# Estadísticas de uso ADATRAP

In [7]:
import csv
import datetime
import os

import pandas as pd
from pytz import timezone as pytz_timezone

project_path = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
data_path = os.path.join(project_path, 'data')

# Evaluar periodos en horario local
location = pytz_timezone('America/Santiago')

In [8]:
# sacar un dump de las estadísticas que se han capturado
# PASOS A SEGUIR
# Ingresar al servidor de producción adatrap.cl
# Ingresar a psql con: python manage.py dbshell
# Copiar datos de log a un archivo csv: \copy (select * from logapp_useractions left join auth_user on logapp_useractions.user_id=auth_user.id) TO '/tmp/datos.csv' DELIMITER ',' CSV HEADER;
# Descargar el archivo mediante ftp en el path $data_path

In [9]:
diccionario_acciones = {
    # vistas
    '/profile/transfers/': '',
    '/profile/odmatrix/': '',
    '/profile/expedition/': '',
    '/profile/stop/': '',
    '/profile/trajectory/': '',

    '/speed/matrix/': '',
    '/speed/variation/': '',
    '/speed/ranking/': '',

    '/globalstat/detail/': '',
    '/globalstat/resume/': '',

    '/trip/strategies/': '',
    '/trip/fromToMaps/': 'vista_origenes_y_destinos',
    '/trip/resume/': 'vista_indicadores_de_viajes',
    '/trip/map/': 'vista_geolocalización_de_viajes',
    '/trip/large-trips/': 'vista_viajes-por_etapas',

    '/shape/base/': '',
    '/shape/map/': '',
    '/shape/route/': '',

    # días disponibles, indica que se cargó alguna vista
    '/esapi/profile/availableRoutes/': 'perfiles_días_disponibles',
    '/esapi/speed/availableDays/': 'velocidades_días_disponibles',
    '/esapi/trip/availableDays/': 'viajes_días_disponibles',
    '/esapi/resume/availableDays/': 'estadísticas_días_disponibles',
    '/esapi/stop/matchedStopData/': 'búsqueda_de_parada',

    # solicitud de datos
    '/esapi/speed/speedByRoute/': 'vista_matriz_velocidades_por_servicio',
    '/esapi/trip/largeTravelData/': 'datos_',
    '/esapi/trip/resumeData/': '',
    '/esapi/profile/loadProfileByStopData/': '',
    '/esapi/profile/availableDays/': '',
    '/esapi/resume/data/': '',
    '/esapi/odbyroute/availableDays/': '',
    '/esapi/odbyroute/availableRoutes/': '',
    '/esapi/trip/mapData/': '',
    '/esapi/speed/availableRoutes/': '',
    '/esapi/speed/speedVariation/': '',
    '/esapi/trip/transfersData/': '',
    '/esapi/trip/fromToMapData/': '',
    '/esapi/trip/strategiesData/': '',
    '/esapi/speed/matrixData/': '',
    '/esapi/profile/loadProfileByExpeditionData/': '',
    '/esapi/speed/rankingData/': '',

    # usuario
    '/user/login/': 'inicio_de_sesión',
    '/user/password_change/': 'vista_cambiar_contraseña',
}


# Cálculo de sesiones

In [10]:
class User:

    def __init__(self, username):
        self.username = username
        self.session_number = 0
        self.last_session = None
        self.max_session_duration = datetime.timedelta()
        self.min_session_duration = datetime.timedelta(days=100)
        self.avg_session_duration = datetime.timedelta()
        self.activities = []

        # tiempo mínimo entre sesiones
        self.time_windows_in_sec = 60 * 15  # 15 minutos

    def add_activity(self, activity):
        self.activities.append(activity)

    def sort_activities(self):
        self.activities.sort(key=lambda x: x[0])

    def calculate_metrics(self):
        self.session_number = 0
        self.last_session = self.activities[0][0]
        start_session = self.activities[0][0]
        previous_timestamp = self.activities[0][0]
        sessions = []

        for index, activity in enumerate(self.activities):
            timestamp = activity[0]

            diff_in_secs = (timestamp - previous_timestamp).total_seconds()
            if diff_in_secs > self.time_windows_in_sec or index == len(self.activities) - 1:
                duration = previous_timestamp.replace(microsecond=0) - start_session.replace(microsecond=0)

                print(self.username, start_session.replace(microsecond=0).astimezone(location),
                      previous_timestamp.replace(microsecond=0).astimezone(location), duration)
                if duration > self.max_session_duration:
                    self.max_session_duration = duration
                if duration < self.min_session_duration:
                    self.min_session_duration = duration
                sessions.append(duration)
                self.last_session = previous_timestamp
                self.session_number += 1
                start_session = timestamp

            previous_timestamp = timestamp

        self.avg_session_duration = sum(sessions, datetime.timedelta()) / len(sessions)
        # quitar microsegundos
        self.avg_session_duration = self.avg_session_duration - datetime.timedelta(
            microseconds=self.avg_session_duration.microseconds)

    def get_results(self):
        fmt = '%Y-%m-%d %H:%M:%S %Z'
        last_session = self.last_session.astimezone(location).strftime(fmt)
        return [self.username, self.session_number, last_session, self.max_session_duration, self.min_session_duration,
                self.avg_session_duration]

# Cargar datos

In [25]:
def build_user_activity(start_time, end_time):
    filename = 'datos.csv'
    input_path = os.path.join(data_path, filename)
    users = dict()

    with open(input_path, encoding='utf-8') as csvfile:
        spamreader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
        next(spamreader)
        url_set = set()
        for row in spamreader:
            # no es relevante lo que hace el administrador
            url = row['url'].split('?')[0]
            try:
                timestamp = datetime.datetime.strptime('{0}{1}'.format(row['timestamp'], '00'),
                                                       '%Y-%m-%d %H:%M:%S.%f%z')
            except ValueError:
                timestamp = datetime.datetime.strptime('{0}{1}'.format(row['timestamp'], '00'), '%Y-%m-%d %H:%M:%S%z')
            username = row['username']

            if not (start_time <= timestamp < end_time):
                continue

            #if url.startswith('/admin'):
            #    continue
            # actividades no relevantes
            if url in ['/favicon.ico', '/']:
                continue

            if username not in users:
                users[username] = User(username)

            users[username].add_activity([timestamp])

    for username in users:
        users[username].sort_activities()

    return users

# Sesiones por usuario

In [26]:
start_time = location.localize(datetime.datetime(2023, 5, 1, 0, 0, 0, ))
end_time = location.localize(datetime.datetime(2023, 7, 1, 0, 0, 0, ))

users = build_user_activity(start_time, end_time)

labels = (
    'usuario', 'n° sesiones', 'última sesión', 'tiempo máximo de sesión', 'tiempo mínimo de sesión', 'sesión promedio')
data = []
for username in users:
    user = users[username]
    user.calculate_metrics()
    data.append(user.get_results())

df = pd.DataFrame(data, columns=labels).sort_values(by=['n° sesiones'], ascending=False)
df = df[df['usuario'] != 'transantiago']
df = df[df['usuario'] != 'dtpm']
df = df[df['usuario'] != 'prueba']

macarena.salazar 2023-05-02 09:26:55-04:00 2023-05-02 09:34:39-04:00 0:07:44
macarena.salazar 2023-05-02 11:47:51-04:00 2023-05-02 11:49:51-04:00 0:02:00
macarena.salazar 2023-05-04 09:18:31-04:00 2023-05-04 09:25:53-04:00 0:07:22
macarena.salazar 2023-05-04 15:05:36-04:00 2023-05-04 15:05:43-04:00 0:00:07
macarena.salazar 2023-05-04 15:29:35-04:00 2023-05-04 15:54:27-04:00 0:24:52
macarena.salazar 2023-05-04 16:12:50-04:00 2023-05-04 16:14:16-04:00 0:01:26
macarena.salazar 2023-05-08 10:37:35-04:00 2023-05-08 10:37:35-04:00 0:00:00
macarena.salazar 2023-05-08 11:54:15-04:00 2023-05-08 11:59:01-04:00 0:04:46
macarena.salazar 2023-05-08 13:11:50-04:00 2023-05-08 13:11:56-04:00 0:00:06
macarena.salazar 2023-05-09 12:14:30-04:00 2023-05-09 12:14:30-04:00 0:00:00
macarena.salazar 2023-05-10 11:22:46-04:00 2023-05-10 11:26:09-04:00 0:03:23
macarena.salazar 2023-05-10 12:44:12-04:00 2023-05-10 12:44:13-04:00 0:00:01
macarena.salazar 2023-05-10 17:37:51-04:00 2023-05-10 17:37:53-04:00 0:00:02

In [27]:
df.style

,usuario,n° sesiones,última sesión,tiempo máximo de sesión,tiempo mínimo de sesión,sesión promedio
2,tomas.rebolledo,217,2023-06-30 13:38:15 -04,0 days 00:57:35,0 days 00:00:00,0 days 00:05:49
8,juan.lagos,216,2023-06-30 12:36:11 -04,0 days 00:50:26,0 days 00:00:00,0 days 00:09:12
15,fabian.perez,149,2023-06-30 16:11:38 -04,0 days 00:47:11,0 days 00:00:00,0 days 00:04:50
0,macarena.salazar,128,2023-06-30 16:44:18 -04,0 days 01:53:10,0 days 00:00:00,0 days 00:06:45
3,miguel.vielma,119,2023-06-30 15:25:29 -04,0 days 00:29:10,0 days 00:00:00,0 days 00:03:13
4,agustin.musante,96,2023-06-30 12:45:47 -04,0 days 00:17:01,0 days 00:00:00,0 days 00:02:27
7,katherine.Garrido,87,2023-06-28 16:40:53 -04,0 days 01:15:08,0 days 00:00:00,0 days 00:10:18
1,sebastian.tamblay,85,2023-06-30 19:45:39 -04,0 days 01:13:40,0 days 00:00:00,0 days 00:06:00
5,andres.gonzalez,74,2023-06-30 12:04:30 -04,0 days 00:32:34,0 days 00:00:00,0 days 00:03:20
19,patrick.oemick,66,2023-06-30 12:40:09 -04,0 days 00:44:01,0 days 00:00:00,0 days 00:03:48


In [29]:
file_path = os.path.join(data_path, 'tabla_de_actividad.csv')
df['última sesión'] = df['última sesión'].apply(
    lambda x: x.replace(' -04', '').replace('0 days ', '').replace(' -03', ''))
df['tiempo máximo de sesión'] = df['tiempo máximo de sesión'].apply(
    lambda x: str(x).replace(' -04', '').replace('0 days ', '').replace(' -03', ''))
df['tiempo mínimo de sesión'] = df['tiempo mínimo de sesión'].apply(
    lambda x: str(x).replace(' -04', '').replace('0 days ', '').replace(' -03', ''))
df['sesión promedio'] = df['sesión promedio'].apply(
    lambda x: str(x).replace(' -04', '').replace('0 days ', '').replace(' -03', ''))

df.to_csv(file_path, index=False, sep=',')

# Post proceso
- abrir el csv
- Pegar el contenido del csv en un google sheet
- Expandir los datos a varias columnas (por defecto se almacenan en una por fila). Para esto hacer lo siguiente:
    - Ir a Data
    - Seleccionar "Split text to columns"
- Incorporar una columna a la izquierda de la tabla con el número de fila y el header "N°"
- Copiar la tabla de google sheet y seleccionar las celdas de la tabla del doc donde irán los valores y seleccionar pegado normal